In [2]:
# first of all, let's check that avro tools work

!avro

Version 1.11.1 of Apache Avro
Copyright 2010-2015 The Apache Software Foundation

This product includes software developed at
The Apache Software Foundation (https://www.apache.org/).
----------------
Available tools:
    canonical  Converts an Avro Schema to its canonical form
          cat  Extracts samples from files
      compile  Generates Java code for the given schema.
       concat  Concatenates avro files without re-compressing.
        count  Counts the records in avro files or folders
  fingerprint  Returns the fingerprint for the schemas.
   fragtojson  Renders a binary-encoded Avro datum as JSON.
     fromjson  Reads JSON records and writes an Avro data file.
     fromtext  Imports a text file into an avro data file.
      getmeta  Prints out the metadata of an Avro data file.
    getschema  Prints out schema of an Avro data file.
          idl  Generates a JSON schema from an Avro IDL file
 idl2schemata  Extract JSON schemata of the types from an Avro IDL file
       indu

In [3]:
from pyhive import hive
from time import sleep


# A simple loop waiting for the Hive to become available
while True:
    try:
        h = hive.Connection(host='hiveserver2', port=10000).cursor()
        h.execute('SHOW SCHEMAS')
        print("")
        print("## HIVE WORKS! Existing schemas: {} ##".format(h.fetchall()))
        break
    except Exception as e:
        print("Waiting 10s for hive to start...")
        sleep(10)


## HIVE WORKS! Existing schemas: [('default',)] ##


In [4]:
# create a test table

h.execute('CREATE TABLE IF NOT EXISTS default.test (id int, name string) STORED AS ORC')
h.execute('SHOW TABLES FROM default')
h.fetchall()

[('test',)]

In [10]:
# insert some data into the test table to verify that the ./_data folder is writable

h.execute('INSERT INTO default.test VALUES (1, "Jane"), (2, "Joe")')
h.execute('SELECT * FROM default.test')
h.fetchall()

OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: SemanticException [Error 10001]: Line 1:12 Table not found 'test':17:16", 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:335', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:199', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:260', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:247', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:541', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:516', 'org.apache.hive.service.cli.CLIService:executeStatement:CLIService.java:282', 'org.apache.hive.service.cli.thrift.ThriftCLIService:ExecuteStatement:ThriftCLIService.java:563', 'org.apache.hive.service.rpc.thrift.TCLIServi

In [8]:
# remove the test table and its data

h.execute('DROP TABLE default.test')
h.execute('SHOW TABLES FROM default')
h.fetchall()

[]

In [9]:
# create Hive schema for iceberg tables

h.execute('CREATE SCHEMA IF NOT EXISTS iceberg')
h.execute('SHOW SCHEMAS')
h.fetchall()

[('default',), ('iceberg',)]